### **<u>Item 71: Prefer deque for Producer-Consumer Queues</u>**

#### <span style="color: yellowgreen">**_(1) FIFO_**</span>

&emsp; 1. **이메일 처리 예시**

In [1]:
# 이메일 메시지를 나타내는 클래스

class Email:
    def __init__(self, sender, receiver, message):
        self.sender = sender
        self.receiver = receiver
        self.message = message
        ...

In [2]:
class NoEmailError(Exception):
    pass

def try_receive_email():
    # Email 인스턴스를 반환하거나 NoEmailError 예외 발생
    ...

In [3]:
def produce_emails(queue):
    while True:
        try:
            email = try_receive_email()
        except NoEmailError:
            return
        else:
            queue.append(email)  # Producer

In [4]:
def consume_one_email(queue):
    if not queue:
        return
    email = queue.pop(0)  # Consumer
    
    # 이메일을 장기 보관을 위해 인덱싱하는 코드
    ...

In [5]:
def loop(queue, keep_running):
    while keep_running():
        produce_emails(queue)
        consume_one_email(queue)

def my_end_func():
    ...
loop([], my_end_func)


In [6]:
import timeit

def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'\n원소 수: {count:>5,} 걸린 시간: {avg_iteration:.6f}초')
    return count, avg_iteration

def list_append_benchmark(count):
    def run(queue):
        for i in range(count):
            queue.append(i)

    tests = timeit.repeat(
        setup ='queue = []',
        stmt = 'run(queue)',
        globals=locals(),
        repeat=  1000,
        number=1)
    
    return print_results(count, tests)

In [7]:
def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'데이터 크기 {growth:>4.1f}배, 걸린 시간 {slowdown:>4.1f}배')

baseline = list_append_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = list_append_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린 시간: 0.000027초

원소 수: 1,000 걸린 시간: 0.000052초
데이터 크기  2.0배, 걸린 시간  1.9배

원소 수: 2,000 걸린 시간: 0.000103초
데이터 크기  4.0배, 걸린 시간  3.8배

원소 수: 3,000 걸린 시간: 0.000157초
데이터 크기  6.0배, 걸린 시간  5.9배

원소 수: 4,000 걸린 시간: 0.000216초
데이터 크기  8.0배, 걸린 시간  8.1배

원소 수: 5,000 걸린 시간: 0.000256초
데이터 크기 10.0배, 걸린 시간  9.6배


#### <span style="color: yellowgreen">**_(2) deque_**</span>

&emsp; **append & appendleft**

In [9]:
# deque 활용의 기본 예시

from collections import deque

mydeq = deque([1,2,3,4])
mydeq.appendleft(-10)        # 왼쪽에 데이터 삽입
print(mydeq)                 # deque([-10, 1, 2, 3, 4])

mydeq.append(10)             # 오른쪽에 데이터 삽입
print(mydeq)                 # deque([-10, 1, 2, 3, 4, 10])

deque([-10, 1, 2, 3, 4])
deque([-10, 1, 2, 3, 4, 10])


&emsp; **pop & popleft**

In [ ]:
from collections import deque

mydeq = deque([1,2,3,4])

# 오른쪽에서 데이터 제거
print(mydeq.pop())          # 4
mydeq                       # deque([1, 2, 3])

# 왼쪽에서 데이터 제거
print(mydeq.popleft())      # 1
mydeq                       # deque([2, 3])

&emsp; **deque관련 참고 메서드**

&emsp;&emsp; - append(item): item을 덱의 오른쪽에 삽입 <br>
&emsp;&emsp; - appendleft(item): Item을 덱의 왼쪽에 삽입<br>
&emsp;&emsp; - pop(): 덱의 오른쪽 끝 요소를 반환하는 동시에 덱에서 제거<br>
&emsp;&emsp; - popleft(): 덱의 왼쪽 끝 요소를 반환하는 동시에 덱에서 제거<br>
&emsp;&emsp; - extend(list): 배열(list)을 순환하면서 덱의 오른쪽에 삽입<br>
&emsp;&emsp; - extendleft(list): 배열(list)을 순환하면서 덱의 왼쪽에 추가<br>
&emsp;&emsp; - emove(item): item을 덱에서 찾아 제거<br>
&emsp;&emsp; - rotate(n): 덱의 n만큼 회전(양수일 경우 오른쪽, 음수일 경우 왼쪽)<br>

&emsp; 2. **이메일 처리 예시에 deque 적용**

In [8]:
import collections
def  consume_one_email(queue):
    if not  queue:
        return
    email = queue.pop(0) #소비자

    #전자우편 메시지 처리
    ...

def  my_end_func():
    ...

loop(collections.deque(), my_end_func)
#append의 성능이 리스트를 사용하는 성능과 거의 비슷

def deque_append_benchmark(count):
    def prepare():
        return collections.deque()

    def run(queue):
        while queue:
            queue.pop(0)

    tests = timeit.repeat(
        setup = 'queue = prepare()',
        stmt ='run(queue)',
        globals = locals(),
        repeat = 1000,
        number =1)
    return print_results(count, tests)
import collections

baseline = deque_append_benchmark(500)

for count in (1_000,2_000,3_000,4_000,5_000):
    comparison = deque_append_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린 시간: 0.000000초

원소 수: 1,000 걸린 시간: 0.000000초
데이터 크기  2.0배, 걸린 시간  1.0배

원소 수: 2,000 걸린 시간: 0.000000초
데이터 크기  4.0배, 걸린 시간  1.0배

원소 수: 3,000 걸린 시간: 0.000000초
데이터 크기  6.0배, 걸린 시간  1.0배

원소 수: 4,000 걸린 시간: 0.000000초
데이터 크기  8.0배, 걸린 시간  1.0배

원소 수: 5,000 걸린 시간: 0.000000초
데이터 크기 10.0배, 걸린 시간  1.0배


#### <span style="color: yellowgreen">**_Summary_**</span>

&emsp; &#9312; 생산자는 append를 호출해 원소를 추가, 소비자는 pop(0)을 사용해 원소를 받음<br><br>
&emsp; &#9313; 리스트를 FIFO queue로 사용 시 queue의 길이가 늘어남에 따라 성능 저하 <br><br>
&emsp; &#9314; deque는 queue의 길이와 관계없이 상수 시간 만에 append와 pop을 수행하기 땜누에 FIFO 구현에 이상적